Linear Regression with PyTorch¶
Part 2 of "PyTorch: Zero to GANs"

This post is the second in a series of tutorials on building deep learning models with PyTorch, an open source neural networks library developed and maintained by Facebook. Check out the full series:

 
 
2. [Linear Regression & Gradient Descent](https://jovian.ml/aakashns/02-linear-regression)
3. [Image Classfication using Logistic Regression](https://jovian.ml/aakashns/03-logistic-regression) 
4. [Training Deep Neural Networks on a GPU](https://jovian.ml/aakashns/04-feedforward-nn)
5. [Image Classification using Convolutional Neural Networks](https://jovian.ml/aakashns/05-cifar10-cnn)
6. [Data Augmentation, Regularization and ResNets](https://jovian.ml/aakashns/05b-cifar10-resnet)
7. [Generating Images using Generative Adverserial Networks](https://jovian.ml/aakashns/06-mnist-gan)

This series attempts to make PyTorch a bit more approachable for people starting out with deep learning and neural networks. In this notebook, we’ll cover the basic building blocks of PyTorch models: tensors and gradients.




https://blog.exxactcorp.com/the-most-important-fundamentals-of-pytorch-you-should-know/

Continuing where the [previous tutorial](https://jvn.io/aakashns/3143ceb92b4f4cbbb4f30e203580b77b) left off, we'll discuss one of the foundational  algorithms of machine learning in this post: *Linear regression*. We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

![linear-regression-graph](https://i.imgur.com/4DJ9f8X.png)

The *learning* part of linear regression is to figure out a set of weights `w11, w12,... w23, b1 & b2` by looking at the training data, to make accurate predictions for new data (i.e. to predict the yields for apples and oranges in a new region using the average temperature, rainfall and humidity). This is done by adjusting the weights slightly many times to make better predictions, using an optimization technique called *gradient descent*.
 

In [95]:
import numpy as np
import torch

 ## Training data

The training data can be represented using 2 matrices: `inputs` and `targets`, each with one row per observation, and one column per variable.

In [96]:
 # Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
print(inputs )
print(inputs[1,1])
print(inputs[1])

[[ 73.  67.  43.]
 [ 91.  88.  64.]
 [ 87. 134.  58.]
 [102.  43.  37.]
 [ 69.  96.  70.]]
88.0
[91. 88. 64.]


In [97]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

We've separated the input and target variables, because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors as follows:

In [98]:
 # Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)
inputs[1]

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


tensor([91., 88., 64.])

## Linear regression model from scratch

The weights and biases (`w11, w12,... w23, b1 & b2`) can also be represented as matrices, initialized as random values. The first row of `w` and the first element of `b` are used to predict the first target variable i.e. yield of apples, and similarly the second for oranges.

In [99]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.6101, -0.2155, -0.0783],
        [-1.1625,  0.2573,  0.8358]], requires_grad=True)
tensor([-0.0799, -0.9864], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [100]:
 def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables. 

In [101]:
 # Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 99.6528, -32.6672],
        [122.4648, -30.6365],
        [106.5823, -19.1637],
        [151.9874, -77.5694],
        [ 84.8487,   2.0110]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.


In [102]:
 # Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


 You can see that there's a huge difference between the predictions of our model, and the actual values of the target variables. Obviously, this is because we've initialized our model with random weights and biases, and we can't expect it to just work.
Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets, using the following method:

    Calculate the difference between the two matrices (preds and targets).
    Square all elements of the difference matrix to remove negative values.
    Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the mean squared error (MSE).


In [103]:
 # MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

torch.sum = sum of all the elements in a tensor, and the 
diff.numel() =  number of elements in a tensor. 
Let's compute the mean squared error for the current predictions of our model.

In [104]:
 # Compute loss
loss = mse(preds, targets)
print(loss) 
 

tensor(9884.0352, grad_fn=<DivBackward0>)


## interpret the result 
On average, each element in the prediction differs from the actual target by about 145 (square root of the loss 20834). And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. Also, the result is called the loss, because it indicates how bad the model is at predicting the target variables. 
Lower the loss, better the model.


In [105]:
 #Compute gradients 
 loss.backward()   

w.r.t = with  respect to
The gradients are stored in the .grad property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix, with the same dimensions.

In [106]:
 # Gradients for weights
print("          w11        w12      w13"    )
print(w)
# w.grad[0,0] is the dervitive of the loss w.r.t w11
print(w.grad)

          w11        w12      w13
tensor([[ 1.6101, -0.2155, -0.0783],
        [-1.1625,  0.2573,  0.8358]], requires_grad=True)
tensor([[  3577.1770,   1751.3210,   1469.9071],
        [-10494.0381, -11002.0146,  -6818.6436]])


In [107]:
 print(b)
 print(b.grad)   

tensor([-0.0799, -0.9864], requires_grad=True)
tensor([  36.9072, -123.6052])


The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the [slope](https://en.wikipedia.org/wiki/Slope) of the loss function w.r.t. the weights and biases.

If a gradient element is **positive**:
* **increasing** the element's value slightly will **increase** the loss.
* **decreasing** the element's value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/hFYoVgU.png)

If a gradient element is **negative**:
* **increasing** the element's value slightly will **decrease** the loss.
* **decreasing** the element's value slightly will **increase** the loss.

![negative=gradient](https://i.imgur.com/w3Wii7C.png)

The increase or decrease in loss by changing a weight element is proportional to the value of the gradient of the loss w.r.t. that element. This forms the basis for the optimization algorithm that we'll use to improve our model.

Before we proceed, we reset the gradients to zero by calling .zero_() method. We need to do this, because PyTorch accumulates, gradients i.e. the next time we call .backward on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results. 

In [108]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


 ## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [109]:
 # Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 99.6528, -32.6672],
        [122.4648, -30.6365],
        [106.5823, -19.1637],
        [151.9874, -77.5694],
        [ 84.8487,   2.0110]], grad_fn=<AddBackward0>)


 Note that the predictions are same as before, since we haven't made any changes to our model. The same holds true for the loss and gradients.


In [110]:
 # Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(9884.0352, grad_fn=<DivBackward0>)


In [111]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[  3577.1770,   1751.3210,   1469.9071],
        [-10494.0381, -11002.0146,  -6818.6436]])
tensor([  36.9072, -123.6052])


Finally, we update the weights and biases using the gradients computed above.

In [112]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

A few things to note above:

    We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate or modify gradients while updating the weights and biases.

    We multiply the gradients with a really small number (10^-5 in this case), to ensure that we don't modify the weights by a really large amount, since we only want to take a small step in the downhill direction of the gradient. This number is called the learning rate of the algorithm.

    After we have updated the weights, we reset the gradients back to zero, to avoid affecting any future computations.

Let's take a look at the new weights and biases.


In [113]:
print(w)
print(b)

tensor([[ 1.5743, -0.2330, -0.0930],
        [-1.0575,  0.3674,  0.9039]], requires_grad=True)
tensor([-0.0803, -0.9851], requires_grad=True)


In [114]:
#With the new weights and biases, the model should have lower loss.
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(7189.9751, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient desce

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.


In [115]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [116]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(484.8320, grad_fn=<DivBackward0>)


In [117]:
# Predictions
preds

tensor([[ 67.1085,  68.6888],
        [ 84.8119, 101.7313],
        [ 96.7826, 133.1469],
        [ 78.5442,  28.7677],
        [ 72.9021, 125.6037]], grad_fn=<AddBackward0>)

In [118]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Linear regression using PyTorch built-ins

The model and training process above were implemented using basic matrix operations. But since this such a common pattern , PyTorch has several built-in functions and classes to make it easy to create and train models.

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.


In [119]:
import torch.nn as nn

In [120]:
 # Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)


We are using 15 training examples this time, to illustrate how to work with large datasets in small batches.


## Dataset and DataLoader

We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.


In [121]:
from torch.utils.data import TensorDataset

In [122]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
print(train_ds[0 ])
train_ds[0:3]

(tensor([73., 67., 43.]), tensor([56., 70.]))


(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.


In [123]:
from torch.utils.data import DataLoader

In [124]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

The data loader is typically used in a for-in loop. Let's look at an example.

In [125]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break
 

tensor([[ 87., 134.,  58.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.]])
tensor([[119., 133.],
        [119., 133.],
        [ 22.,  37.],
        [ 22.,  37.],
        [ 56.,  70.]])


In each iteration, the data loader returns one batch of data, with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, which can lead to faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.


In [126]:
# Define model  1:11 https://www.youtube.com/watch?v=1HWwDUCxGFU
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)
print(model)

Parameter containing:
tensor([[ 0.4997, -0.3861,  0.3926],
        [-0.2925, -0.0245, -0.2690]], requires_grad=True)
Parameter containing:
tensor([ 0.2592, -0.2970], requires_grad=True)
Linear(in_features=3, out_features=2, bias=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.


In [127]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.4997, -0.3861,  0.3926],
         [-0.2925, -0.0245, -0.2690]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2592, -0.2970], requires_grad=True)]

In [128]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 27.7521, -34.8616],
        [ 36.8837, -46.2914],
        [ 14.7672, -44.6346],
        [ 49.1557, -41.1424],
        [ 25.1562, -41.6661],
        [ 27.7521, -34.8616],
        [ 36.8837, -46.2914],
        [ 14.7672, -44.6346],
        [ 49.1557, -41.1424],
        [ 25.1562, -41.6661],
        [ 27.7521, -34.8616],
        [ 36.8837, -46.2914],
        [ 14.7672, -44.6346],
        [ 49.1557, -41.1424],
        [ 25.1562, -41.6661]], grad_fn=<AddmmBackward>)

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function mse_loss.


In [129]:
# Import nn.functional
import torch.nn.functional as F
#The nn.functional package contains many useful loss functions and several other utilities.

In [130]:
# Define loss function
loss_fn = F.mse_loss

In [131]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(11657.0342, grad_fn=<MseLossBackward>)


In [132]:
# ? F.mse_loss

## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD stands for stochastic gradient descent. It is called stochastic because samples are selected in batches (often with random shuffling) instead of as a single group.


In [133]:
# Define optimizer
# lr = learning rate
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD, so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate which controls the amount by which the parameters are modified.


## Train the model

We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

    Generate predictions

    Calculate the loss

    Compute gradients w.r.t the weights and biases

    Adjust the weights by subtracting a small quantity proportional to the gradient

    Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function fit which trains the model for a given number of epochs.


In [136]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

    We use the data loader defined earlier to get batches of data for every iteration.

    Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update, and opt.zero_grad to reset the gradients to zero.

    We've also added a log statement which prints the loss from the last batch of data for every 10th epoch, to track the progress of training. loss.item returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.


In [137]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss: 979.6053
Epoch [20/100], Loss: 710.6135
Epoch [30/100], Loss: 226.2627
Epoch [40/100], Loss: 240.2098
Epoch [50/100], Loss: 67.4182
Epoch [60/100], Loss: 52.0228
Epoch [70/100], Loss: 78.3614
Epoch [80/100], Loss: 35.7488
Epoch [90/100], Loss: 33.9918
Epoch [100/100], Loss: 25.7199


In [138]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.9886,  71.3671],
        [ 83.4535,  97.1244],
        [113.1822, 138.7065],
        [ 31.2314,  43.5070],
        [ 98.1977, 109.1680],
        [ 58.9886,  71.3671],
        [ 83.4535,  97.1244],
        [113.1822, 138.7065],
        [ 31.2314,  43.5070],
        [ 98.1977, 109.1680],
        [ 58.9886,  71.3671],
        [ 83.4535,  97.1244],
        [113.1822, 138.7065],
        [ 31.2314,  43.5070],
        [ 98.1977, 109.1680]], grad_fn=<AddmmBackward>)

In [139]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])